In [1]:
import os
import pandas as pd
import xml.etree.ElementTree as ET
from pathlib import Path
from shutil import copy2

# === Step 1: Buat metadata.csv dari struktur 8-dataset ===
base_dir = "8-dataset"
splits = ["train", "val", "test"]
os.makedirs(base_dir, exist_ok=True)

data = []
for split in splits:
    image_dir = os.path.join(base_dir, split, "images")
    if not os.path.exists(image_dir):
        continue
    for filename in os.listdir(image_dir):
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            parts = Path(filename).stem.split("_")
            country = parts[1] if len(parts) > 2 else "Unknown"
            data.append({
                "filename": filename,
                "country": country,
                "split": split
            })

metadata_df = pd.DataFrame(data)
metadata_csv_path = os.path.join(base_dir, "metadata.csv")
metadata_df.to_csv(metadata_csv_path, index=False)


In [16]:
# === Step 2: Konversi XML ke format YOLOv8 ===
output_root = "8-dataset-kaggle"
class_map = {"D40": 0}  # hanya pothole

for split in ["test", "train", "val"]:
    os.makedirs(os.path.join(output_root, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(output_root, split, "labels"), exist_ok=True)

    image_dir = os.path.join(base_dir, split, "images")
    annot_dir = os.path.join(base_dir, split, "annotations")

    for image_file in os.listdir(image_dir):
        if not image_file.endswith(('.jpg', '.jpeg', '.png')):
            continue
        image_path = os.path.join(image_dir, image_file)
        annot_path = os.path.join(annot_dir, Path(image_file).stem + ".xml")
        label_path = os.path.join(output_root, split, "labels", Path(image_file).stem + ".txt")
        dest_image_path = os.path.join(output_root, split, "images", image_file)

        copy2(image_path, dest_image_path)

        if not os.path.exists(annot_path):
            open(label_path, 'w').close()
            continue

        tree = ET.parse(annot_path)
        root = tree.getroot()
        size = root.find("size")
        w = float(size.find("width").text)
        h = float(size.find("height").text)

        lines = []
        for obj in root.findall("object"):
            cls = obj.find("name").text
            if cls not in class_map:
                continue
            label = class_map[cls]
            bndbox = obj.find("bndbox")
            xmin = float(bndbox.find("xmin").text)
            ymin = float(bndbox.find("ymin").text)
            xmax = float(bndbox.find("xmax").text)
            ymax = float(bndbox.find("ymax").text)

            x_center = (xmin + xmax) / 2.0 / w
            y_center = (ymin + ymax) / 2.0 / h
            bw = (xmax - xmin) / w
            bh = (ymax - ymin) / h

            lines.append(f"{label} {x_center:.6f} {y_center:.6f} {bw:.6f} {bh:.6f}")

        with open(label_path, 'w') as f:
            f.write("\n".join(lines))

In [29]:
import os
print("CPU cores:", os.cpu_count())

CPU cores: 16


In [1]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # Bisa ganti ke yolov8s.pt, yolov8m.pt, dll

model.train(
    data="data.yaml",
    epochs=50,
    imgsz=640,
    batch=32,
    name="yolov8n-D40-50-2",
    device=0,
)

New https://pypi.org/project/ultralytics/8.3.141 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.135  Python-3.10.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n-D40-50-2, nbs=64, nms=False, opset=None, op

train: Scanning D:\Pothole Vision - AI Road Damage Detection\8-dataset-yolo\train\labels.cache... 6014 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6014/6014 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.00.0 ms, read: 1409.1716.2 MB/s, size: 98.9 KB)


val: Scanning D:\Pothole Vision - AI Road Damage Detection\8-dataset-yolo\val\labels.cache... 1472 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1472/1472 [00:00<?, ?it/s]


Plotting labels to d:\Pothole Vision - AI Road Damage Detection\runs\detect\yolov8n-D40-50-2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to d:\Pothole Vision - AI Road Damage Detection\runs\detect\yolov8n-D40-50-2
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       1472       2951      0.752      0.648       0.73      0.375



50 epochs completed in 2.453 hours.
Optimizer stripped from d:\Pothole Vision - AI Road Damage Detection\runs\detect\yolov8n-D40-50-2\weights\last.pt, 6.2MB
Optimizer stripped from d:\Pothole Vision - AI Road Damage Detection\runs\detect\yolov8n-D40-50-2\weights\best.pt, 6.2MB

Validating d:\Pothole Vision - AI Road Damage Detection\runs\detect\yolov8n-D40-50-2\weights\best.pt...
Ultralytics 8.3.135  Python-3.10.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:09<00:00,  2.36it/s]


                   all       1472       2951      0.754      0.647       0.73      0.375
Speed: 0.2ms preprocess, 2.8ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to d:\Pothole Vision - AI Road Damage Detection\runs\detect\yolov8n-D40-50-2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000207D3A37EE0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [ ]:
yolo task=detect mode=predict model="runs/detect/yolov8n-D40/weights/best.pt" source="8-dataset-yolo/test/images" save=True save_txt=True


In [25]:
import cv2
from ultralytics import YOLO

# Load model YOLOv8 yang sudah kamu train (ganti path modelmu)
model_path = 'runs/detect/yolov8n-D40/weights/best.pt'  # contoh path
model = YOLO(model_path)

# Buka webcam (device 0 biasanya webcam bawaan laptop)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Tidak bisa membuka webcam")
    exit()

print("Tekan 'q' untuk keluar...")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Gagal membaca frame dari webcam")
        break

    # Jalankan deteksi YOLOv8 pada frame webcam
    results = model(frame)

    # Ambil frame hasil yang sudah diberi kotak deteksi
    annotated_frame = results[0].plot()

    # Tampilkan frame
    cv2.imshow('YOLOv8 Webcam Detection', annotated_frame)

    # Tekan 'q' untuk keluar
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Bersihkan dan tutup
cap.release()
cv2.destroyAllWindows()


Tekan 'q' untuk keluar...

0: 480x640 1 Pothole, 8.0ms
Speed: 1.2ms preprocess, 8.0ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Pothole, 10.0ms
Speed: 1.1ms preprocess, 10.0ms inference, 2.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Pothole, 4.5ms
Speed: 1.1ms preprocess, 4.5ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Pothole, 4.5ms
Speed: 1.1ms preprocess, 4.5ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Pothole, 5.6ms
Speed: 1.1ms preprocess, 5.6ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Pothole, 4.6ms
Speed: 1.1ms preprocess, 4.6ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Pothole, 4.6ms
Speed: 1.3ms preprocess, 4.6ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Pothole, 7.9ms
Speed: 1.0ms preprocess, 7.9ms inference, 1.3ms postprocess per imag

In [15]:
import pandas as pd
from pathlib import Path
from tabulate import tabulate

# Folder dasar
base_path = Path("runs/detect")

# Kolom yang dibutuhkan
columns_needed = [
    'epoch',
    'metrics/precision(B)',
    'metrics/recall(B)',
    'metrics/mAP50(B)',
    'metrics/mAP50-95(B)',
    'val/box_loss',
    'val/cls_loss',
    'val/dfl_loss',
]

# Kumpulkan data
data_summary = []

for csv_file in base_path.rglob("results.csv"):
    try:
        df = pd.read_csv(csv_file)
        last_row = df.iloc[-1][columns_needed]
        model_name = csv_file.parent.name
        data_summary.append([model_name] + last_row.values.tolist())
    except Exception as e:
        print(f"Error reading {csv_file}: {e}")

# Tampilkan tabel rapi
headers = ["Model"] + columns_needed
print(tabulate(data_summary, headers=headers, tablefmt="grid", floatfmt=".4f"))


+------------------+----------+------------------------+---------------------+--------------------+-----------------------+----------------+----------------+----------------+
| Model            |    epoch |   metrics/precision(B) |   metrics/recall(B) |   metrics/mAP50(B) |   metrics/mAP50-95(B) |   val/box_loss |   val/cls_loss |   val/dfl_loss |
+==================+==========+========================+=====================+====================+=======================+================+================+================+
| yolov8m-D40-50   |  50.0000 |                 0.7898 |              0.6482 |             0.7451 |                0.3891 |         1.7401 |         1.1283 |         1.6480 |
+------------------+----------+------------------------+---------------------+--------------------+-----------------------+----------------+----------------+----------------+
| yolov8n-D40-100  | 100.0000 |                 0.7729 |              0.6656 |             0.7336 |                0.3825 |  